In [203]:
b.matlab_eng.slm_close(nargout=0)
b.matlab_eng.camera_close(nargout=0)
b.matlab_eng.quit()

In [ ]:
import numpy as np
import data1D
from reservoir import Reservoir
from sklearn.model_selection import GridSearchCV
import data_utils
import itertools

%load_ext autoreload
%autoreload 2
%aimport data1D
%aimport reservoir

%matplotlib inline
import matplotlib.pyplot as plt

import matlab.engine

In [187]:
# Input data
n_res=5000
n_sequence=1
spatial_points = 30
spatial_points_ = spatial_points
lyap_exp = 0.0461
n_train = 3000
n_test = 2000
forget = 400
parallel_runs = 1
ks_data_origin, xx, tt = data1D.kuramoto_sivashinsky_from_dataset(
    sequence_length=n_train+n_test, n_sequence=n_sequence, spatial_points=spatial_points)

In [38]:
# Matlab engine initialization
matlab_eng = matlab.engine.start_matlab()
matlab_eng.cd(r'D:\projects\reservoir-computing-python\hardware_control', nargout=0)

# Camera initialization
cam_roi = [200, 200]
cam_sampling_range = np.linspace(0, (cam_roi[0]-1)*(cam_roi[1]-1)-1, n_res, dtype='uint32')
if n_res > (cam_roi[0]-1)*(cam_roi[1]-1)-1:
    warnings.warn("The number of camera pixels is less than the required size of the reservoir")
matlab_eng.workspace['cam_roi'] = matlab.double(cam_roi)
matlab_eng.camera_open(nargout=0)

# SLM initialization
slm_size = [1920, 1152]
matlab_eng.workspace['slm_size'] = matlab.double(slm_size)
matlab_eng.slm_open(nargout=0)

In [188]:
ks_data_train = ks_data_origin[:, :n_train, :]
input_shape = ks_data_train.shape

ks_data_test = ks_data_origin[:, n_train:, :]

In [191]:
params ={
    'activation_param1': np.linspace(100, 150, 6),
    'leak_rate': np.linspace(0, 0.5, 6),
    'train_param': np.linspace(0.1, 0.6, 6)}

In [193]:
for x in itertools.product(*params.values()):

    b = Reservoir(n_res=n_res, res_scale=1, res_encoding='meadowlark slm', res_enc_param=128,
                  remove_saturated_neurons=True, n_saturated_neurons=20, # reservoir cleanup 
                  input_scale=1, input_encoding='meadowlark slm', input_enc_param=128,
                  random_projection='meadowlark slm',
                  scale_input_MinMax = (0, 1), scale_output_MinMax = (0, 1), # data standardization
                  raw_input_feature = True, enc_input_feature = False, # concatenated states properties
                  activation_fun='intensity_in_tanh', activation_param = 300, activation_param1 = 130, activation_param2 = 3.5,
                  parallel_runs=parallel_runs,  bias_scale=np.sqrt(2), leak_rate=0.25,
                  pred_horizon=1, rec_pred_steps=200, forget = forget,
                  train_method='ridge', train_param=0.45, verbose=0,
                  cam_roi=cam_roi, cam_sampling_range=cam_sampling_range, 
                  slm_size=slm_size, matlab_eng=matlab_eng, # SLM experiment
                  random_state = 100
                 )

    for param, value in zip(params, x):
        setattr(b, param, value)
        print(''.join([str(param),' = ', str(value)]))

    # Algorithm fitting
    b.fit(ks_data_train);

    # Algorithm testing
    b.parallel_test_runs=10
    score = b.predict_and_score(ks_data_test, only_score=True)
    print(''.join(['score = ', str(score)]))
    print('-----------------------------------------------------')

activation_param1 = 100.0
leak_rate = 0.0
train_param = 0.1
score = 0.48141277340098887
-----------------------------------------------------
activation_param1 = 100.0
leak_rate = 0.0
train_param = 0.2
score = 0.4893656166392739
-----------------------------------------------------
activation_param1 = 100.0
leak_rate = 0.0
train_param = 0.30000000000000004
score = 0.4505674661979021
-----------------------------------------------------
activation_param1 = 100.0
leak_rate = 0.0
train_param = 0.4
score = 0.6358113689843162
-----------------------------------------------------
activation_param1 = 100.0
leak_rate = 0.0
train_param = 0.5
score = 0.6491200483699362
-----------------------------------------------------
activation_param1 = 100.0
leak_rate = 0.0
train_param = 0.6
score = 0.5955793101738414
-----------------------------------------------------
activation_param1 = 100.0
leak_rate = 0.1
train_param = 0.1
score = 0.5570687966097644
--------------------------------------------------

activation_param1 = 110.0
leak_rate = 0.30000000000000004
train_param = 0.30000000000000004
score = 0.6689202547138722
-----------------------------------------------------
activation_param1 = 110.0
leak_rate = 0.30000000000000004
train_param = 0.4
score = 0.4786326670664518
-----------------------------------------------------
activation_param1 = 110.0
leak_rate = 0.30000000000000004
train_param = 0.5
score = 0.3459398684993443
-----------------------------------------------------
activation_param1 = 110.0
leak_rate = 0.30000000000000004
train_param = 0.6
score = 0.4115995432587567
-----------------------------------------------------
activation_param1 = 110.0
leak_rate = 0.4
train_param = 0.1
score = 0.45159515936609174
-----------------------------------------------------
activation_param1 = 110.0
leak_rate = 0.4
train_param = 0.2
score = 0.45521629636412353
-----------------------------------------------------
activation_param1 = 110.0
leak_rate = 0.4
train_param = 0.30000000000000

MatlabExecutionError: 
  File D:\projects\reservoir-computing-python\hardware_control\get_speckle.m, line 44, in get_speckle
Missing trigger, no frame captured by the camera.
